# Clonamos el repositorio para obtener los dataSet

In [ ]:
!git clone https://github.com/joanby/machinelearning-az.git

# Damos acceso a nuestro Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Test it

In [ ]:
!ls '/content/drive/My Drive' 

#Google colab tools

In [ ]:
from google.colab import files # Para manejar los archivos y, por ejemplo, exportar a su navegador
import glob # Para manejar los archivos y, por ejemplo, exportar a su navegador
from google.colab import drive # Montar tu Google drive

# Plantilla de Pre Procesado

# Cómo importar las librerías


In [1]:
import numpy as np
import matplotlib.pyplot as plt # usaremos la sublibrería de matplotlib, para representación gráfica
import pandas as pd # manejo de datos

# Importar el data set


In [48]:
dataset = pd.read_csv('Data.csv') # ver en que carpeta está (poner DB en misma carpeta que el notebook)
X = dataset.iloc[:, :-1].values # variables independientes (todas las filas, todas las columnas menos la última) MATRIZ
y = dataset.iloc[:, 3].values # variable dependiente a predecir (todas las filas, la 4ta columna ) VECTOR
# iloc me toma las filas y columnas por posición, mientras que loc por nombre
# Columnas y filas empiezan en el índice 0
# En spyder, variable explorer podemos ver los dataframes y cambiar el formato de número, por si está en notación científica (poner %.0f en format)

# Tratamiento de los NAs

In [3]:
# SÓLO CUANDO TENGA NAN
from sklearn.impute import SimpleImputer

In [49]:
imputer = SimpleImputer(missing_values = np.nan, strategy = "mean") # podemos reemplazar por la mediana, moda o valor constante (poner en el primero la forma en que se aparecen los valores nulos)
imputer = imputer.fit(X[:,1:3]) # Ajusta o detecta los valores nan. No puedo aplicar la técnica a todas las columnas, sólo a las numéricas
X[:, 1:3] = imputer.transform(X[:,1:3]) # Reemplazamos los valores nan por las medias de la columna en donde se encuentran (en todas las filas, pero sólo desde la 2da hasta la 3ra columna)
# El último número luego de los : no toma hasta el índice de esa columna, sino hasta la anterior
# Axis = 0 es para columnas, Axis = 1 es para filas

In [50]:
result = pd.DataFrame({'Column1': X[:, 0], 'Column2': X[:, 1],'Column3': X[:, 2]})
print(X.shape) # filas, columnas

(10, 3)


In [51]:
display(result)

,Column1,Column2,Column3
0,France,44.0,72000.0
1,Spain,27.0,48000.0
2,Germany,30.0,54000.0
3,Spain,38.0,61000.0
4,Germany,40.0,63777.777778
5,France,35.0,58000.0
6,Spain,38.777778,52000.0
7,France,48.0,79000.0
8,Germany,50.0,83000.0
9,France,37.0,67000.0


# Codificar datos categoricos

In [37]:
# SOLO CUANDO TENGA DATOS CATEGÓRICOS
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [52]:
labelencoder_X = LabelEncoder() # creamos un codificador de datos
X[:, 0] = labelencoder_X.fit_transform(X[:, 0]) # tomo la columna 0 y la paso a datos numéricos (codifica del 0 al 2 según orden alfabético), reemplazando en el array de los datos
# Pero las categorias ordinales en este problema no me sirven, ya que no hay grado de comparación entre paises (debemos pasar a dummys)

In [53]:
# Con one-hot encoding, vamos a pasar a tener tantas columnas como etiquetas que tiene dicha categoría (dummy)
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [0])],   # Los números de columnas a ser transformados (ponemos [0] pero puedee ser [0, 1, 3]) (hacer también para [3])
    remainder='passthrough'    # Dejamos el resto de las columnas sin modificar                 
)

In [54]:
X = np.array(ct.fit_transform(X), dtype=np.float64)
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y) # transformamos los (Si,No) en (1,0). No hace falta aplicar el One Hot Encoder al ser 2 clases que pueden ser ordenadas (ausencia o presencia de atributo)


In [56]:
result = pd.DataFrame({'Column1(Francia)': X[:, 0], 'Column2(Alemania)': X[:, 1],'Column3(España)': X[:, 2], 'Age': X[:, 3],'Salary': X[:, 4],'Purchased': y[:]})
display(result)

,Column1(Francia),Column2(Alemania),Column3(España),Age,Salary,Purchased
0,1.0,0.0,0.0,44.000000,72000.000000,0
1,0.0,0.0,1.0,27.000000,48000.000000,1
2,0.0,1.0,0.0,30.000000,54000.000000,0
3,0.0,0.0,1.0,38.000000,61000.000000,0
4,0.0,1.0,0.0,40.000000,63777.777778,1
5,1.0,0.0,0.0,35.000000,58000.000000,1
6,0.0,0.0,1.0,38.777778,52000.000000,0
7,1.0,0.0,0.0,48.000000,79000.000000,1
8,0.0,1.0,0.0,50.000000,83000.000000,0
9,1.0,0.0,0.0,37.000000,67000.000000,1


# Dividir el data set en conjunto de entrenamiento y conjunto de testing


In [57]:
# SE HACE SIEMPRE
from sklearn.model_selection import train_test_split

In [60]:
# Dividimos los datos en train/test (70/30 u 80/20) para que el modelo no memorice todos los datos de entrenamiento, sino que encuentre los patrones subyacentes en los datos que le permitan formar carácter predictivo sobre la variable dependiente, y luego elabore predicciones sobre la parte que no se usó para entrenar (ahí corroboramos su eficaacia)
# Evitar caer en el Overffiting o Sobreajuste del modelo a los datos de entrenamiento, ya que el mismo por aprenderselo bien pierde capacidad de predicción frente a nuevos datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
# X y Y train, variables para entrenar el algoritmo; X_test variables brindadas al algoritmo para que elabore una predicción; Y_test para corroborar la efectividad de dicha predicción
# random_state es una semilla que permite que los resultados de la división aleatoria de los datos, y por consiguiente de la efectividad del modelo, no sea tan aleatoria y se repita
# según la semilla que usemos, tendremos diferentes datos en train y en test

# Escalado de variables

In [61]:
# SE HACE LA MAYORÍA DE VECES
from sklearn.preprocessing import StandardScaler

In [62]:
# Aplicamos estandarización (para establecer un mismo rango de valores de las variables, y de paso tender a Gauss). Soluciona el problema de la distancia euclidea
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train) # escalamos automáticamente a los datos de train
X_test = sc_X.transform(X_test) # se acopla al escalado establecido en train (solo transform)
print(X_train.shape)
print(X_test.shape)
# Las variables dummy pueden ser escaladas o no, dependiendo el criterio de cada uno
# Los árboles de decisión no utilizan distancia euclidea, pero la estandarización/normalización hace que el modelo converga mucho más rápido
# y_train, y_test no los reescalamos porque nuestro algoritmo es de clasificación, sin embargo, cuando sea de predicción numérica, debemos normalizar también el conjunto con el mismo transform usado en x 

(8, 5)
(2, 5)


In [63]:
result_Xtrain = pd.DataFrame({'Column1': X_train[:, 0], 'Column2': X_train[:, 1],'Column3': X_train[:, 2], 'Column4': X_train[:, 3],'Column5': X_train[:, 4]})

In [64]:
display(result_Xtrain)

,Column1,Column2,Column3,Column4,Column5
0,-1.0,2.645751,-0.774597,0.263068,0.123815
1,1.0,-0.377964,-0.774597,-0.253501,0.461756
2,-1.0,-0.377964,1.290994,-1.975398,-1.530933
3,-1.0,-0.377964,1.290994,0.052614,-1.111420
4,1.0,-0.377964,-0.774597,1.640585,1.720297
5,-1.0,-0.377964,1.290994,-0.081312,-0.167514
6,1.0,-0.377964,-0.774597,0.951826,0.986148
7,1.0,-0.377964,-0.774597,-0.597881,-0.482149


In [65]:
result_X_test = pd.DataFrame({'Column1': X_test[:, 0], 'Column2': X_test[:, 1],'Column3': X_test[:, 2], 'Column4': X_test[:, 3],'Column5': X_test[:, 4]})

In [66]:
display(result_X_test)

,Column1,Column2,Column3,Column4,Column5
0,-1.0,2.645751,-0.774597,-1.458829,-0.901663
1,-1.0,2.645751,-0.774597,1.984964,2.139811
